<a href="https://colab.research.google.com/github/kiranraou/Python-Projects/blob/main/CRF_POS_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Sequence Learning - Assignment1

**Problem Statement 2.**

Provided with a corpus that has a lot of sentences with POS 
tagging completed (CRF_POS_dataset.csv). The objective is to use 
this dataset and build a Condition Random Field (CRF model) with 
sequence modeling

**Dataset Description.** 

Dataset: CRF_POS_dataset.csv

CRF_POS_dataset contains a lot of sentences.

**Mounting Google Drive.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from collections import Counter

In [5]:
data = pd.read_csv("/content/drive/MyDrive/NIT Warangal _Industry Project/CRF_POS_dataset.csv")
data.head(50)

,word,pos
0,Thousands,NNS
1,of,IN
2,demonstrators,NNS
3,have,VBP
4,marched,VBN
5,through,IN
6,London,NNP
7,to,TO
8,protest,VB
9,the,DT


In [6]:
sentences=[]
i=0
while i<len(data):
    tmp=[]
    while (data.iloc[i,0])!="." and data.iloc[i,1] !='.':
        tmp.append(data.iloc[i,0])
        i=i+1
    tmp.append('.')
    sentences.append(np.array(tmp))
    i=i+1

In [7]:
tagged_sentences=[]
i=0
while i<len(data):
    tmp=[]
    while (data.iloc[i,0])!="." and data.iloc[i,1] !='.':
        tmp.append(data.iloc[i,1])
        i=i+1
    tmp.append('.')
    tagged_sentences.append(np.array(tmp))
    i=i+1

In [8]:
print(len(tagged_sentences)),
    
print(len(sentences))

13690
13690


In [9]:
train_sentences, test_sentences, train_tags, test_tags = train_test_split(sentences, tagged_sentences, test_size=0.2)

**Features from text.**

In [10]:
def features(sentence,index):
    return {
        'is_first_capital':int(sentence[index][0].isupper()),
        'is_first_word': int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'is_complete_capital': int(sentence[index].upper()==sentence[index]),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'prefix_1':sentence[index][0],
        'prefix_2': sentence[index][:2],
        'prefix_3':sentence[index][:3],
        'prefix_4':sentence[index][:4],
        'suffix_1':sentence[index][-1],
        'suffix_2':sentence[index][-2:],
        'suffix_3':sentence[index][-3:],
        'suffix_4':sentence[index][-4:],
        'word_has_hyphen': 1 if '-' in sentence[index] else 0
        
        
    }

In [11]:
def prepareData(tagged_sentences):
    X=[]
    for sentences in tagged_sentences:
        X.append([features(sentences, index) for index in range(len(sentences))])
    return X

In [12]:
X_train = prepareData(train_sentences)
y_train = train_tags
X_test = prepareData(test_sentences)
y_test =  test_tags

**CRF Model.**

In [13]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.01, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

*Predictions.*

In [14]:
X_train[1]

[{'is_alphanumeric': 0,
  'is_complete_capital': 0,
  'is_first_capital': 1,
  'is_first_word': 1,
  'is_last_word': 0,
  'is_numeric': 0,
  'next_word': 'soldiers',
  'prefix_1': 'I',
  'prefix_2': 'Is',
  'prefix_3': 'Isr',
  'prefix_4': 'Isra',
  'prev_word': '',
  'suffix_1': 'i',
  'suffix_2': 'li',
  'suffix_3': 'eli',
  'suffix_4': 'aeli',
  'word_has_hyphen': 0},
 {'is_alphanumeric': 0,
  'is_complete_capital': 0,
  'is_first_capital': 0,
  'is_first_word': 0,
  'is_last_word': 0,
  'is_numeric': 0,
  'next_word': 'opened',
  'prefix_1': 's',
  'prefix_2': 'so',
  'prefix_3': 'sol',
  'prefix_4': 'sold',
  'prev_word': 'Israeli',
  'suffix_1': 's',
  'suffix_2': 'rs',
  'suffix_3': 'ers',
  'suffix_4': 'iers',
  'word_has_hyphen': 0},
 {'is_alphanumeric': 0,
  'is_complete_capital': 0,
  'is_first_capital': 0,
  'is_first_word': 0,
  'is_last_word': 0,
  'is_numeric': 0,
  'next_word': 'fire',
  'prefix_1': 'o',
  'prefix_2': 'op',
  'prefix_3': 'ope',
  'prefix_4': 'open',
  '

In [15]:
y_train[1]

array(['JJ', 'NNS', 'VBD', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNS', 'VBG',
       'IN', 'DT', 'JJ', 'NN', 'IN', 'NNP', ',', 'VBG', 'CD', 'IN', 'PRP',
       '.'], dtype='<U3')

In [16]:
y_pred=crf.predict(X_test)
y_pred_train=crf.predict(X_train)

**F1 Score.**

In [17]:
#F1 score test
metrics.flat_f1_score(y_test, y_pred,average='weighted',labels=crf.classes_)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.9806388562072053

In [18]:
#F1 score train
metrics.flat_f1_score(y_train, y_pred_train,average='weighted',labels=crf.classes_)

0.9970536359029604

**Accuracy Score.**

In [19]:
#Accuracy score test
metrics.flat_accuracy_score(y_test,y_pred)

0.9806596297170607

In [20]:
#Accuracy score train
metrics.flat_accuracy_score(y_train,y_pred_train)

0.9970536779158823

**Classification Report.**

In [21]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=crf.classes_, digits=3
))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         NNP      0.975     0.987     0.981      7375
         VBD      0.955     0.973     0.964      2198
          JJ      0.952     0.954     0.953      4478
          NN      0.974     0.970     0.972      8329
          IN      0.993     0.994     0.993      6860
          DT      0.999     0.997     0.998      5635
         NNS      0.986     0.988     0.987      4267
         PRP      0.998     0.993     0.995       808
          TO      0.990     0.997     0.993      1295
         VBN      0.960     0.941     0.950      1833
           ,      0.999     1.000     1.000      1803
         WRB      1.000     0.992     0.996       128
         VBZ      0.984     0.976     0.980      1408
          RB      0.967     0.956     0.961      1154
           .      1.000     1.000     1.000      2738
         VBG      0.964     0.962     0.963      1099
          CD      0.998     0.992     0.995      1262
         POS      0.998    